# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [1]:
%run ../config/bootstrap.py

In [2]:
import pandas as pd
from utils import get_project_root, save_parquet_in_chunks , load_partitioned_parquet
import basedosdados as bd

In [3]:
project_root = get_project_root() 
#billing_id = 'ufrgs-ppgc-dengue'

In [4]:
#%load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100
%sql bigquery://ufrgs-ppgc-dengue

Connecting to 'bigquery://ufrgs-ppgc-dengue'

# Datasets

In [6]:
%%sql sinan <<
SELECT ano
,id_municipio_notificacao as id_municipio
,id_estabelecimento as id_cnes
,  sum(CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  sum(CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  sum(CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano>2020
group by 1,2,3


Running query in 'bigquery://ufrgs-ppgc-dengue'

149469 rows affected.

In [7]:
sinan.head()

,ano,id_municipio,id_cnes,target,obito,flag_caso_encerrado,qt
0,2024,1200336,2000083,0,0,144,144
1,2024,1200252,2001330,0,0,25,25
2,2024,1600303,2020599,0,0,899,900
3,2024,2704302,2720035,0,0,188,188
4,2024,1302603,4727118,0,0,47,48


In [8]:
path = project_root / "data/4_gold/dim_cnes/dim_cnes_part_1.parquet"
cnes = pd.read_parquet(path)
cnes.head()

,id_cnes,id_unidade_cnes,nu_cnpj_mantenedora,id_pfpj,nivel_dep,no_razao_social,no_fantasia,no_logradouro,nu_endereco,no_complemento,no_bairro,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,nu_telefone,nu_fax,no_email,nu_cpf,nu_cnpj,id_atividade,id_clientela,nu_alvara,data_expedicao,id_orgao_expedidor,data_val_lic_sani,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,data_atualizacao,id_usuario,id_cpfdiretorcln,reg_diretorcln,st_adesao_filantrop,id_motivo_desab,no_url,nu_latitude,nu_longitude,data_atualizacao_geolocalizacao,no_usuario_geo,id_natureza_jur,id_estab_sempre_aberto,st_geracredito_gerente_sgif,st_conexao_internet,id_tipo_unidade,no_fantasia_abrev,id_gestao,data_atualizacao_origem,id_tipo_estabelecimento,id_atividade_principal,st_contrato_formalizado,id_tipo_abrangencia,st_coworking,hash,data_inicio_vigencia_silver,data_fim_vigencia_silver,data_atualizacao_silver,vigencia_silver
0,4013484,2408104013484,08241754000145,3,3,RIO GRANDE DO NORTE SECRETARIA DA SAUDE PUBLICA,HOSPITAL GISELDA TRIGUEIRO,CONEGO MONTE,110,None,QUINTAS,59037170,7,None,04,None,32327915,None,None,None,08241754010965,03,03,None,None,None,None,None,05,06,24,240810,2025-08-07 00:00:00,SAUDE,68051085353,4363,None,None,None,"-5,794974298299524","-35,209068059921265",2021-12-06 00:00:00,4013484,1023,S,None,S,None,None,D,2003-10-30 00:00:00,006,009,S,None,N,4c98b01f2fd50a2da5a7fcb49580d51674ee156797054d93dc8b3b301afade8b,2025-08-17,9999-12-31,2025-08-17,1
1,2589648,5201402589648,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APDA GO,UPA BRASICON,RUA DAS ACASIA,S/N,ESQ C AV CHILE,RESIDENCIAL BRASICON,74975390,001,None,None,None,3545 9953,None,None,None,None,04,03,None,None,None,None,None,73,06,52,520140,2024-07-26 00:00:00,YASMIN,03258865124,None,None,None,None,"-16,82760","-49,24209",2024-07-26 00:00:00,YASMIN,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,None,None,None,f70e972e8d235757c25415e536078cb9b4e8e1b93b4d07280c2e961b8154df77,2025-08-17,9999-12-31,2025-08-17,1
2,2052474,3549802052474,None,3,1,UNIMED SJRPRETO COOPERATIVA DE TRABALHO MEDICO,COMPLEXO DE SAUDE UNIMED SAO JOSE DO RIO PRETO,AVENIDA BADY BASSITT,4870,None,JAD ALTO RIO PRETO,15025000,0215,None,None,None,17 3202-1110,None,unimed@unimedripreto.com.br,None,45100138000362,04,03,354980501-861-001673-1-2,2024-09-13 00:00:00,2,2025-09-13 00:00:00,1,73,06,35,354980,2025-06-04 00:00:00,FERNANDA,25123834826,None,None,None,None,"-20,819074142767942","-49,397743074317205",2025-04-01 00:00:00,FERNANDA,2143,S,None,S,None,None,M,2003-02-25 00:00:00,008,007,N,None,None,66339a46703b49c1bab3c25113d169321010c222f882e1944b11b9d5363cae9d,2025-08-17,9999-12-31,2025-08-17,1
3,6270093,3549806270093,46588950000180,3,3,PREFEITURA MUNICIPAL DE SAO JOSE DO RIO PRETO,UPA REGIAO NORTE,R MANOEL MORENO ESQ R JOSEPHA VOLTARELLI SANFELICE,130,None,JARDIM ANTUNES,15047050,0215,None,None,None,17-32375315,None,None,None,None,03,03,354980501-861-002357-1-7,2018-02-23 00:00:00,2,2019-02-23 00:00:00,1,73,06,35,354980,2025-08-08 00:00:00,FERNANDA,21728064805,473613,None,None,None,"-20,777456","-49,403220",2019-10-01 00:00:00,FERNANDA,1244,S,None,S,None,None,M,2009-06-23 00:00:00,008,007,None,None,None,245706442b1794984d4002593ef3ea47d4e082ca8ca5a07ffbe07421cdc06207,2025-08-17,9999-12-31,2025-08-17,1
4,2589613,5201402589613,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APARECIDA DE GOIANIA,CAIS NOVA ERA,AVENIDA SAO JOAO,S/N,None,JARDIM NOVA ERA,74916970,001,None,None,None,(62)3545 5925,None,None,None,None,04,03,None,2024-01-31 00:00:00,2,2024-12-31 00:00:00,1,20,06,52,520140,2025-01-28 00:00:00,YASMIN,01162515147,None,None,None,None,"-16,7474832","-49,2812776",2024-02-01 00:00:00,MARCIA,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,S,None,None,603e683c06ba7c1292ed6742e66b4bbc3e50da43cafb25e931b6f944e1324839,2025-08-17,9999-12-31,2025-08-17,1


In [9]:
sinan_cnes = sinan.merge(cnes, on="id_cnes", how="left")
sinan_cnes = sinan_cnes.drop_duplicates()

In [10]:
sinan_cnes.head()

,ano,id_municipio,id_cnes,target,obito,flag_caso_encerrado,qt,id_unidade_cnes,nu_cnpj_mantenedora,id_pfpj,nivel_dep,no_razao_social,no_fantasia,no_logradouro,nu_endereco,no_complemento,no_bairro,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,nu_telefone,nu_fax,no_email,nu_cpf,nu_cnpj,id_atividade,id_clientela,nu_alvara,data_expedicao,id_orgao_expedidor,data_val_lic_sani,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,data_atualizacao,id_usuario,id_cpfdiretorcln,reg_diretorcln,st_adesao_filantrop,id_motivo_desab,no_url,nu_latitude,nu_longitude,data_atualizacao_geolocalizacao,no_usuario_geo,id_natureza_jur,id_estab_sempre_aberto,st_geracredito_gerente_sgif,st_conexao_internet,id_tipo_unidade,no_fantasia_abrev,id_gestao,data_atualizacao_origem,id_tipo_estabelecimento,id_atividade_principal,st_contrato_formalizado,id_tipo_abrangencia,st_coworking,hash,data_inicio_vigencia_silver,data_fim_vigencia_silver,data_atualizacao_silver,vigencia_silver
0,2024,1200336,2000083,0,0,144,144,1200332000083,04034526000143,3,3,SECRETARIA DE ESTADO DE SAUDE,HOSPITAL DR ABEL PINHEIRO MACIEL FILHO,JAPIIM,S/N,None,CENTRO,69990000,003,None,None,None,None,None,None,None,04034526002006,04,03,None,None,None,None,None,05,06,12,120033,2025-05-09 00:00:00,MESTRE,55824501220,None,None,None,None,"-7,6158847","-72,9101281",2019-08-28 00:00:00,MESTRE,1023,S,None,S,None,None,E,2003-05-13 00:00:00,006,009,None,None,N,661ea68353772dbc157bff65e8a6bb1f3ad4781153aa9b89b95039341db1723b,2025-08-17,9999-12-31,2025-08-17,1.0
1,2024,1200252,2001330,0,0,25,25,1200252001330,84306588000104,3,3,PREFEITURA MUNICIPAL DE EPITACIOLANDIA,CENTRO DE SAUDE JOSE CANDIDO DE MESQUITA,RUA DOM JULIO MATIOLLE,S/N,None,CENTRO,69934000,002,None,None,None,(68)05463986,None,None,None,None,04,03,None,None,None,None,None,02,03,12,120025,2025-06-05 00:00:00,SCNES,73768871215,None,None,None,None,"-11,0231800","-68,458600",2019-09-23 00:00:00,SCNES,1244,N,None,N,None,None,M,2001-10-30 00:00:00,001,012,None,03,N,5899ab3f2a1668aa7e7ad655158f69ddb51599b9d3dfa0162b5509f9dfaa57e8,2025-08-17,9999-12-31,2025-08-17,1.0
2,2024,1600303,2020599,0,0,899,900,1600302020599,05995766000177,3,3,MUNICIPIO DE MACAPA,PMM U B S DR MARCELO CANDIA,VEREADOR JULIO MARIA PINTO PEREIRA,1329,None,JARDIM FELICIDADE I,68909000,001,None,None,None,(96)2131115,None,None,None,None,04,03,None,None,None,None,None,02,06,16,160030,2025-07-22 00:00:00,SCNES,82397279215,None,None,None,None,"0,0824155","-51,0642088",2025-04-23 00:00:00,SCNES,1244,S,None,S,None,None,M,2001-11-01 00:00:00,001,012,S,None,N,b5410d9943169c92d210f3c128ce51afa064e50170c7af4847bc16a91068a6a7,2025-08-17,9999-12-31,2025-08-17,1.0
3,2024,2704302,2720035,0,0,188,188,2704302720035,12517793000108,3,3,UNIVERSIDADE ESTADUAL DE CIENCIAS DA SAUDE DE ALAGOAS,HOSPITAL ESCOLA DR HELVIO AUTO,CONEGO FERNANDO LYRA,S/N,None,TRAPICHE,57010430,None,None,None,None,None,None,None,None,12517793000876,03,03,None,None,None,None,None,07,06,27,270430,2025-07-28 00:00:00,GESTOR,00972439498,None,None,None,None,"-9,660746235997152","-35,700974464416504",2025-07-28 00:00:00,GESTOR,1112,S,None,S,None,None,E,2003-09-03 00:00:00,006,009,None,None,N,f49f2784ac404f7ff23b777b877df5dac42afc3ab61464d2231f0d194608e677,2025-08-17,9999-12-31,2025-08-17,1.0
4,2024,1302603,4727118,0,0,47,48,1302604727118,None,3,1,HAPVIDA ASSISTENCIA MEDICA S A,HOSPITAL RIO SOLIMOES,ALVARO MAIA,1131,None,ADRIANOPOLIS,69057035,None,None,None,None,85 40023633,None,ADRIANYSB@HAPVIDA.COM.BR,None,63554067014652,04,03,AMP2300077780,2023-06-29 00:00:00,2,2031-06-29 00:00:00,1,05,04,13,130260,2025-07-24 00:00:00,RIOSOLIMOES,57482667172,3312,None,None,None,"-3,0831372","-60,0359488",2024-07-24 00:00:00,AMCNES,2054,S,None,S,None,None,E,2024-08-04 00:00:00,006,009,N,01,N,19b39ad331274b622800051a5a7ec57cab61032455e0fb7db67590b562b59baf,2025-08-17,9999-12-31,2025-08-17,1.0


In [11]:
sinan_cnes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149469 entries, 0 to 149468
Data columns (total 67 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   ano                              149469 non-null  int64         
 1   id_municipio                     149469 non-null  object        
 2   id_cnes                          149424 non-null  object        
 3   target                           149469 non-null  int64         
 4   obito                            149469 non-null  int64         
 5   flag_caso_encerrado              149469 non-null  int64         
 6   qt                               149469 non-null  int64         
 7   id_unidade_cnes                  147476 non-null  object        
 8   nu_cnpj_mantenedora              123058 non-null  object        
 9   id_pfpj                          147476 non-null  object        
 10  nivel_dep                        147476 non-

In [12]:
output = project_root / "data/4_gold/dim_cnes_w_notifications/dim_cnes_w_notifications.parquet"
save_parquet_in_chunks(sinan_cnes,output,100,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/149469 [00:00<?, ?linhas/s]

✔️ dim_cnes_w_notifications.parquet_part_1.parquet salvo com 14.53 MB (149469 linhas)
✅ Todos os arquivos salvos com sucesso.
